In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tensorflow==2.18.0
!pip install keras==3.8.0
!pip install scikit-learn==1.6.1
!pip install numpy==2.0.2
!pip install pandas==2.2.2

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import pickle

In [5]:
# Load dataset
df = pd.read_csv("/content/drive/MyDrive/lstm deeplearning training/updated_dataset.csv")

In [6]:
# Encode 'label' (Crop Name)
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["label"])

In [7]:
# Save label encoder
with open("label_encoderfinal.pkl", "wb") as file:
    pickle.dump(label_encoder, file)

In [8]:
# One-hot encode 'Location' (to keep text in X)
df = pd.get_dummies(df, columns=["Location"], dtype=int)

In [9]:
# Normalize numerical features
scaler = MinMaxScaler()
X = scaler.fit_transform(df.drop(columns=["label"]))
y = np.array(df["label"])

In [10]:
# Save scaler
with open("scalerfinal.pkl", "wb") as file:
    pickle.dump(scaler, file)

In [11]:
# Reshape for LSTM
X = X.reshape((X.shape[0], 1, X.shape[1]))

In [12]:
# Define LSTM model
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
    Dropout(0.2),
    LSTM(32, return_sequences=False),
    Dropout(0.2),
    Dense(16, activation="relu"),
    Dense(len(label_encoder.classes_), activation="softmax")  # Output layer
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [13]:
# Compile model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])


In [14]:

# Train model
model.fit(X, y, epochs=65, batch_size=16, validation_split=0.2)

Epoch 1/65
110/110 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.0570 - loss: 3.0735 - val_accuracy: 0.0000e+00 - val_loss: 3.8736
Epoch 2/65
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0693 - loss: 2.8995 - val_accuracy: 0.0000e+00 - val_loss: 5.2447
Epoch 3/65
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.1833 - loss: 2.4469 - val_accuracy: 0.0000e+00 - val_loss: 9.7204
Epoch 4/65
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.4163 - loss: 1.7883 - val_accuracy: 0.0000e+00 - val_loss: 13.1247
Epoch 5/65
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5267 - loss: 1.3499 - val_accuracy: 0.0114 - val_loss: 14.7025
Epoch 6/65
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6349 - loss: 1.0921 - val_accuracy: 0.0205 - val_loss: 15.5318
Epoch 7/65
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6718 - loss: 0.9153 - val_accuracy: 0.0432 - val_loss: 16.4629
Epoch 8/65
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7298 - loss: 0.75

In [15]:
# Save model
model.save("lstm_crop_modelfinal.h5")
print("Model saved as lstm_crop_modelfinal.h5")

Model saved as lstm_crop_modelfinal.h5


In [16]:
import pandas as pd
import numpy as np
import tensorflow as tf
import pickle

# Load trained model
model = tf.keras.models.load_model("lstm_crop_modelfinal.h5")

# Load encoders
with open("label_encoderfinal.pkl", "rb") as file:
    label_encoder = pickle.load(file)

with open("scalerfinal.pkl", "rb") as file:
    scaler = pickle.load(file)

# Extract the original training feature names
original_features = scaler.feature_names_in_

# Function to preprocess new input data
def preprocess_input(data):
    # Convert input to DataFrame
    new_data = pd.DataFrame([data])

    # One-hot encode 'Location' (create all possible location columns)
    new_data = pd.get_dummies(new_data, columns=["Location"])

    # Add missing one-hot encoded location columns with value 0
    for col in original_features:
        if col not in new_data.columns:
            new_data[col] = 0  # Add missing column with default 0

    # Reorder columns to match training order
    new_data = new_data[original_features]

    # Normalize input
    new_data = scaler.transform(new_data)

    # Reshape for LSTM
    return new_data.reshape((1, 1, new_data.shape[1]))

# Function to predict crop
def predict_crop(input_data):
    processed_data = preprocess_input(input_data)
    pred = model.predict(processed_data)
    predicted_crop = label_encoder.inverse_transform([np.argmax(pred)])
    return predicted_crop[0]

# Example input (including Latitude & Longitude)
input_data = {
    "N": 27, "P": 76, "K": 83, "temperature": 19.12829388,
    "humidity": 14.92241479, "ph": 6.289614016, "rainfall": 89.61857826,
    "Latitude": 26.23011, "Longitude": 92.94331,  # Included Latitude & Longitude
    "Location": "Assam"  # Text format
}

# Get prediction
predicted_crop = predict_crop(input_data)
print("Predicted Crop:", predicted_crop)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 314ms/step
Predicted Crop: chickpea
